In [13]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [14]:
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader

nvida_doc = PyPDFLoader('docs/nvidia-document.pdf')
nvida_doc2 = PyMuPDFLoader('docs/nvidia-document.pdf')

In [15]:
data = nvida_doc.load()
print(data[0].page_content)

Table of Contents
UNITED ST ATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________________________________________________________________
FORM 10-K
☒ ANNUAL  REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
    For the fiscal year ended January 28, 2024
OR
☐ TRANSITION REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
Commission file number: 0-23985
 
  
NVIDIA CORPORATION
(Exact name of registrant as specified in its charter)
Delaware 94-3177549
(State or other jurisdiction of (I.R.S. Employer
incorporation or organization) Identification No.)
   2788 San T omas Expressway , Santa Clara, California95051
  (Address of principal executive offices) (Zip Code)
Registrant’s telephone number, including area code: (408) 486-2000
Securities registered pursuant to Section 12(b) of the Act:
Title of each class Trading Symbol(s) Name of each exchange on which registered
Common Stock, $0.001 p

In [16]:
print(data[0].metadata)

{'source': 'docs/nvidia-document.pdf', 'page': 0}


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1700,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(len=100,model="gpt-3.5-turbo", )
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(len=100,model="gpt-3.5-turbo", )
nvidia_doc_chunks = text_splitter.split_documents(data)

In [18]:
nvidia_doc_chunks

[Document(page_content='Table of Contents\nUNITED ST ATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________________________________________________________\nFORM 10-K\n☒ ANNUAL  REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n    For the fiscal year ended January 28, 2024\nOR\n☐ TRANSITION REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nCommission file number: 0-23985\n \n  \nNVIDIA CORPORATION\n(Exact name of registrant as specified in its charter)\nDelaware 94-3177549\n(State or other jurisdiction of (I.R.S. Employer\nincorporation or organization) Identification No.)\n   2788 San T omas Expressway , Santa Clara, California95051\n  (Address of principal executive offices) (Zip Code)\nRegistrant’s telephone number, including area code: (408) 486-2000\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class Trading Symbol(s) Name of each excha

In [19]:
max_chunk_length = 0

for chunk in nvidia_doc_chunks:
  max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

1362


In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


system_template = "You are an financial expert from nvidia"
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

In [21]:
openai_gpt_3_5_turbo_model = ChatOpenAI(model="gpt-3.5-turbo")

chain = chat_prompt | openai_gpt_3_5_turbo_model


print(chain.invoke({"content": "Who is the E-VP, Operations - and how old are they?"}))
print(chain.invoke({"content": "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"}))

content="I'm sorry, but I am unable to provide specific information about individuals, including their age. If you have any other questions about Nvidia or general financial topics, I'd be happy to help."
content="I'm sorry, but as a financial expert from NVIDIA, I do not have access to the specific financial data for Total Amortizable Intangible Assets for Jan 29, 2023. This information would typically be found in the company's financial statements or disclosures. I recommend checking the latest financial reports or contacting the company's investor relations department for this specific information."


In [22]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vector_store = FAISS.from_documents(nvidia_doc_chunks, embedding=embeddings)

In [23]:
retriever = vector_store.as_retriever()

In [24]:
retrieved_documents = retriever.invoke("What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?")
print(retrieved_documents)

[Document(page_content='Table of Contents\nNVIDIA  Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements\n(Continued)\nNote 7 - Amortizable Intangible Assets\nThe components of our amortizable intangible assets are as follows:\n Jan 28, 2024 Jan 29, 2023\n Gross\nCarrying\nAmountAccumulated\nAmortizationNet \nCarrying\nAmountGross\nCarrying\nAmountAccumulated\nAmortizationNet \nCarrying\nAmount\n (In millions)\nAcquisition-related intangible\nassets (1) $ 2,642 $ (1,720) $ 922 $ 3,093 $ (1,614) $ 1,479 \nPatents and licensed technology 449 (259) 190 446 (249) 197 \nTotal intangible assets $ 3,091 $ (1,979) $ 1,112 $ 3,539 $ (1,863) $ 1,676 \n(1)    During the first quarter of fiscal year 2023, we commenced amortization of a $630 million in-process research and development intangible asset related to our acquisition of\nMellanox.\nAmortization expense associated with intangible assets for fiscal years 2024, 2023, and 2022 was $614 million, $699 million, and $563 m

In [25]:
retrieved_documents2 = retriever.invoke("Who is the E-VP, Operations - and how old are they?")
print(retrieved_documents2[0].page_content)

Table of Contents
Our support is enhanced during times of crisis, such as war or economic volatility , to take care of our existing team of world-class talent and their families.
Diversity , Inclusion, and Belonging
We believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees.
When recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in the
technology field, including women, Black/African American, and Hispanic/Latino candidates.
To this end, we have been:
•Partnering with institutions and professional organizations serving historically underrepresented communities;
•Embedding dedicated recruiting teams to busin ess areas to shepherd underrepresented candidates through the interview process and find internal
opportunities;
•Supporting the development of women employees through programs aimed at building a pipeline of future lead

In [26]:
for doc in retrieved_documents:
    print(doc)

page_content='Table of Contents\nNVIDIA  Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements\n(Continued)\nNote 7 - Amortizable Intangible Assets\nThe components of our amortizable intangible assets are as follows:\n Jan 28, 2024 Jan 29, 2023\n Gross\nCarrying\nAmountAccumulated\nAmortizationNet \nCarrying\nAmountGross\nCarrying\nAmountAccumulated\nAmortizationNet \nCarrying\nAmount\n (In millions)\nAcquisition-related intangible\nassets (1) $ 2,642 $ (1,720) $ 922 $ 3,093 $ (1,614) $ 1,479 \nPatents and licensed technology 449 (259) 190 446 (249) 197 \nTotal intangible assets $ 3,091 $ (1,979) $ 1,112 $ 3,539 $ (1,863) $ 1,676 \n(1)    During the first quarter of fiscal year 2023, we commenced amortization of a $630 million in-process research and development intangible asset related to our acquisition of\nMellanox.\nAmortization expense associated with intangible assets for fiscal years 2024, 2023, and 2022 was $614 million, $699 million, and $563 million, re

In [27]:
from langchain import hub
retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [28]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


In [29]:
from langchain.prompts import ChatPromptTemplate


template = """
you can only answer questions related to what's in the context. If it's not in the context, then you would reply with 
'Sorry I have no answer to your particular question. I can only answer things regarding: {context}'

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [30]:
from operator import  itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI()

# retrieval_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )

# retrieval_chain = (
#     {"context": itemgetter("question") | retriever, "question": RunnablePassthrough()}
#     | prompt
#     | model
#     | {"context":itemgetter("context"), "question": itemgetter("question")})


retrieval_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | model, "context": itemgetter("context")}
)

#     from langchain.chains import create_retrieval_chain

# retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)


In [31]:
retrieval_chain.invoke({"question":"What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"})

{'response': AIMessage(content="Sorry I have no answer to your particular question. I can only answer things regarding: \n\n[Document(page_content='Table of Contents\\nNVIDIA  Corporation and Subsidiaries\\nNotes to the Consolidated Financial Statements\\n(Continued)\\nNote 7 - Amortizable Intangible Assets\\nThe components of our amortizable intangible assets are as follows:\\n Jan 28, 2024 Jan 29, 2023\\n Gross\\nCarrying\\nAmountAccumulated\\nAmortizationNet \\nCarrying\\nAmountGross\\nCarrying\\nAmountAccumulated\\nAmortizationNet \\nCarrying\\nAmount\\n (In millions)\\nAcquisition-related intangible\\nassets (1) $ 2,642 $ (1,720) $ 922 $ 3,093 $ (1,614) $ 1,479 \\nPatents and licensed technology 449 (259) 190 446 (249) 197 \\nTotal intangible assets $ 3,091 $ (1,979) $ 1,112 $ 3,539 $ (1,863) $ 1,676 \\n(1)    During the first quarter of fiscal year 2023, we commenced amortization of a $630 million in-process research and development intangible asset related to our acquisition of\

In [32]:
retrieval_chain.invoke({"question":"Who is the E-VP, Operations - and how old are they?"})

{'response': AIMessage(content='Sorry I have no answer to your particular question. I can only answer things regarding: [Document(page_content="Table of Contents\\nOur support is enhanced during times of crisis, such as war or economic volatility , to take care of our existing team of world-class talent and their families.\\nDiversity , Inclusion, and Belonging\\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees.\\nWhen recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in the\\ntechnology field, including women, Black/African American, and Hispanic/Latino candidates.\\nTo this end, we have been:\\n•Partnering with institutions and professional organizations serving historically underrepresented communities;\\n•Embedding dedicated recruiting teams to busin ess areas to shepherd underrepresented candidates through th

In [193]:
%pip install -qU ragas

Note: you may need to restart the kernel to use updated packages.


In [194]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [195]:
len(nvidia_doc_chunks)

96

In [196]:
generator = TestsetGenerator.with_openai()

distributions = {
    simple: 0.3,
    multi_context: 0.4,
    reasoning: 0.3
}

testset = generator.generate_with_langchain_docs(nvidia_doc_chunks, 10, distributions,raise_exceptions=False)

embedding nodes:   0%|          | 0/250 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [197]:
test_df = testset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,episode_done
0,What assumptions are used to estimate the fair...,[Table of Contents\nNVIDIA Corporation and Su...,The assumptions used to estimate the fair valu...,simple,True
1,What is the estimated future amortization expe...,[Table of Contents\nNVIDIA Corporation and Su...,The estimated future amortization expense rela...,simple,True
2,What programs and benefits does NVIDIA offer t...,[Table of Contents\nor generate enough renewab...,NVIDIA offers comprehensive benefits to suppor...,simple,True
3,"""What assumptions are used in estimating the f...",[Table of Contents\nNVIDIA Corporation and Su...,The assumptions used in estimating the fair va...,multi_context,True
4,How do foreign exchange rate changes affect in...,[Table of Contents\nfinancial statement risk a...,If an adverse 10% foreign exchange rate change...,multi_context,True


In [199]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

print(test_questions)
print(test_groundtruths)

['What assumptions are used to estimate the fair value of shares issued under the ESPP?', 'What is the estimated future amortization expense related to the net carrying amount of intangible assets as of January 28, 2024?', 'What programs and benefits does NVIDIA offer to support the compensation, benefits, and well-being of its employees?', '"What assumptions are used in estimating the fair value of shares issued under the ESPP using the Black-Scholes model, and how do they relate to equity awards?"', 'How do foreign exchange rate changes affect income before taxes for balance sheet hedging contracts, and what is the specific impact for January 28, 2024 and January 29, 2023?', 'How does NVIDIA prioritize energy efficiency and diversity and inclusion?', 'Why is it important to assess risks related to climate impacts, environmental sustainability regulations, and their potential impacts on operations, supply chain, and business continuity?', 'What are the risks and obligations of process

In [200]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

print(answers)

['The assumptions used to estimate the fair value of shares issued under the ESPP include the weighted average expected life, risk-free interest rate, volatility, and dividend yield.', 'The estimated future amortization expense related to the net carrying amount of intangible assets as of January 28, 2024, is $1,112 million.', "NVIDIA offers comprehensive benefits to support its employees' physical health, well-being, and financial health. These programs include 401(k) programs in the U.S., statutory and supplemental pension programs outside the U.S., an employee stock purchase program, flexible work hours, and time off policies to address mental health, stress, and time-management challenges. NVIDIA evaluates its benefit offerings globally and aims to provide comparable support across the regions where it operates. Additionally, the company is committed to providing tailored benefits based on the needs of its Community Resource Groups and continuing support for parents, including new 

In [201]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

response_dataset[0]

{'question': 'What assumptions are used to estimate the fair value of shares issued under the ESPP?',
 'answer': 'The assumptions used to estimate the fair value of shares issued under the ESPP include the weighted average expected life, risk-free interest rate, volatility, and dividend yield.',
 'contexts': ['Table of Contents\nNVIDIA  Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements\n(Continued)\nThe fair value of shares issued under our ESPP  have been estimated with the following assumptions:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\n(Using the Black-Scholes model)\nESPP\nWeighted average expected life (in years) 0.1-2.0 0.1-2.0 0.1-2.0\nRisk-free interest rate 3.9%-5.5% —%-4.6% —%-0.5%\nVolatility 31%-67% 43%-72% 20%-58%\nDividend yield 0.1% 0.1% 0.1%\nFor ESPP shares, the expected term represents the average term from the first day of the offering period to the purchase date. The risk-free interest rate\nassumption used to value ESPP  shar

In [202]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [203]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [204]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What assumptions are used to estimate the fair...,The assumptions used to estimate the fair valu...,[Table of Contents\nNVIDIA Corporation and Su...,The assumptions used to estimate the fair valu...,1.000000,1.000000,1.0,1.0,0.999035
1,What is the estimated future amortization expe...,The estimated future amortization expense rela...,[Table of Contents\nNVIDIA Corporation and Su...,The estimated future amortization expense rela...,1.000000,1.000000,1.0,1.0,0.999875
2,What programs and benefits does NVIDIA offer t...,NVIDIA offers comprehensive benefits to suppor...,[Table of Contents\nor generate enough renewab...,NVIDIA offers comprehensive benefits to suppor...,0.666667,0.964778,1.0,1.0,0.577370
3,"""What assumptions are used in estimating the f...",The assumptions used in estimating the fair va...,[Table of Contents\nNVIDIA Corporation and Su...,The assumptions used in estimating the fair va...,1.000000,0.941672,1.0,1.0,0.743835
4,How do foreign exchange rate changes affect in...,Foreign exchange rate changes for balance shee...,[Table of Contents\nNVIDIA Corporation and Su...,If an adverse 10% foreign exchange rate change...,1.000000,0.935159,1.0,1.0,0.616932
5,How does NVIDIA prioritize energy efficiency a...,NVIDIA prioritizes energy efficiency through G...,[Table of Contents\nor generate enough renewab...,NVIDIA prioritizes energy efficiency through t...,0.714286,0.916601,1.0,1.0,0.704695
6,Why is it important to assess risks related to...,"Assessing risks related to climate impacts, en...",[Table of Contents\nWe are monitoring the impa...,It is important to assess risks related to cli...,1.000000,0.970658,1.0,1.0,0.376365
7,What are the risks and obligations of processi...,Sorry I have no answer to your particular ques...,[Table of Contents\nregulator or other industr...,The risks and obligations of processing sensit...,NaN,0.867402,1.0,1.0,NaN
8,What was the percentage increase in operating ...,The percentage increase in operating income in...,[Table of Contents\nThe following table sets f...,Operating income in FY2024 increased by 681% c...,NaN,1.000000,1.0,1.0,0.239540
9,How does NVIDIA attract and retain talented em...,NVIDIA attracts and retains talented employees...,[Table of Contents\nor generate enough renewab...,NVIDIA attracts and retains talented employees...,0.666667,0.980935,1.0,1.0,0.300035


In [207]:
results

{'faithfulness': 0.8810, 'answer_relevancy': 0.9577, 'context_recall': 1.0000, 'context_precision': 1.0000, 'answer_correctness': 0.6175}

In [208]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=model)

In [209]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(model, retrieval_qa_prompt)

In [210]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [211]:
response = retrieval_chain.invoke({"input": "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"})

In [212]:
print(response["answer"])

The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023, was $3,539 million.


In [213]:
response = retrieval_chain.invoke({"input": "Who is the E-VP, Operations - and how old are they?"})

In [214]:
print(response["answer"])

Debora Shoquist is the Executive Vice President of Operations mentioned in the context. She is 69 years old.


In [215]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [216]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [94]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are the potential risks and challenges as...,Potential risks and challenges associated with...,[Table of Contents\nCompute & Networking segme...,Acquisitions or investments involve significan...,1.0,0.996996,1.0,0.679167,0.647609
1,What are some challenges that arise during the...,Some challenges that arise during the integrat...,"[Table of Contents\nharmed, future acquisition...",Some challenges that arise during the integrat...,1.0,0.996379,1.0,1.000000,0.998052
2,What is the purpose of over-the-air software u...,The purpose of over-the-air software updates i...,[Table of Contents\nWe are working with severa...,The purpose of over-the-air software updates i...,1.0,1.000000,1.0,0.833333,0.741426
3,How could acquisitions or investments impact o...,Acquisitions or investments could impact our s...,"[Table of Contents\nharmed, future acquisition...",Acquisitions or investments could impair our a...,1.0,0.913040,1.0,1.000000,0.403420
4,What is the Company's policy on reimbursement ...,Covered Officers are not entitled to indemnifi...,[COMPENSATION RECOVERY POLICY\nLast updated: N...,nan,1.0,0.000000,0.0,0.000000,0.170013
5,What licensing requirements were announced by ...,"In October 2023, the USG announced new and upd...","[Table of Contents\nproducts, and it is likely...",The USG announced new and updated licensing re...,1.0,0.877252,1.0,1.000000,0.744133
6,What is the impact of recent accounting pronou...,The impact of recent accounting pronouncements...,[Table of Contents\nNVIDIA Corporation and Su...,The impact of recent accounting pronouncements...,NaN,0.808711,1.0,1.000000,0.376954
7,What are the risks and challenges of business ...,Some of the risks and challenges of business a...,"[Table of Contents\nharmed, future acquisition...",Acquisitions or investments involve significan...,1.0,0.958028,1.0,1.000000,0.624893
8,How does the concentration of sales impact rev...,The concentration of sales to a limited number...,"[Table of Contents\nharmed, future acquisition...",The concentration of sales can adversely affec...,1.0,0.910736,1.0,1.000000,0.899698
9,Which cloud platforms support access to intera...,"NVIDIA's cloud platforms, such as vGPU for ent...",[Table of Contents\nAt the foundation of the N...,Our cloud platforms such as vGPU for enterpris...,1.0,0.879635,1.0,1.000000,0.745906


In [219]:
results

{'faithfulness': 0.8810, 'answer_relevancy': 0.9577, 'context_recall': 1.0000, 'context_precision': 1.0000, 'answer_correctness': 0.6175}

In [220]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.880952,1.000000,0.119048
1,answer_relevancy,0.957720,0.834078,-0.123643
2,context_recall,1.000000,0.900000,-0.100000
3,context_precision,1.000000,0.851250,-0.148750
4,answer_correctness,0.617520,0.635210,0.017690
